## Course Tuning
### 23

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import lightgbm as lgb
import optuna.integration.lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

### 管理番号

In [2]:
model_No = 23
sample_No = 1

### sub-modelを使用するかどうか

In [3]:
use_sub_model = False

In [4]:
if use_sub_model:
    ALL_MERGE = 'intermediate/{}/all_merge_{}_{}_sub.f'.format(model_No, model_No, sample_No)
else:
    ALL_MERGE = 'intermediate/{}/all_merge_{}_{}.f'.format(model_No, model_No, sample_No)

In [5]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 190)

In [6]:
if use_sub_model:
    all_pitch['predict_curve'] = all_pitch['predict_curve'] / all_pitch['predict_straight']
    all_pitch['predict_slider'] = all_pitch['predict_slider'] / all_pitch['predict_straight']
    all_pitch['predict_shoot'] = all_pitch['predict_shoot'] / all_pitch['predict_straight']
    all_pitch['predict_fork'] = all_pitch['predict_fork'] / all_pitch['predict_straight']
    all_pitch['predict_changeup'] = all_pitch['predict_changeup'] / all_pitch['predict_straight']
    all_pitch['predict_sinker'] = all_pitch['predict_sinker'] / all_pitch['predict_straight']
    all_pitch['predict_cutball'] = all_pitch['predict_cutball'] / all_pitch['predict_straight']
    all_pitch.drop(columns=[
        'predict_0', 'predict_1', 'predict_2', 'predict_3', 'predict_4', 'predict_5', 'predict_6',
        'predict_7', 'predict_8', 'predict_9', 'predict_10', 'predict_11', 'predict_12',
       'predict_straight'
    ], inplace=True)

### train
- 行数: 257117

In [7]:
train = all_pitch.dropna(subset=['course'])
train.shape

(257117, 190)

#### 特徴量に加えた部分を抜く

In [8]:
if sample_No == 1:
    train = train.query('index > 60000')
elif sample_No == 2:
    train = train.query('index <= 60000 | index > 120000')
elif sample_No == 3:
    train = train.query('index <= 120000 | index > 180000')
elif sample_No == 4:
    train = train.query('index <= 180000')
train.shape

(197116, 190)

In [9]:
del all_pitch
gc.collect()

33

In [10]:
train_d = train.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

### LGBM学習
- multiclass : クラス数=13

In [11]:
X_train, X_test, y_train, y_test = train_test_split(train_d, train['course'])
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [12]:
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 13,
    }

In [13]:
%%time
best_params, tuning_history = dict(), list()
lgb_model = lgb.train(lgb_param, lgb_train,
                      valid_sets=lgb_eval,
                      verbose_eval=0,
                      best_params=best_params,
                      tuning_history=tuning_history)
print('Best Params:', best_params)
#print('Tuning history:', tuning_history)

/home/yasunobu_umehara/anaconda3/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning: train is experimental (supported from v0.18.0). The interface can change in the future.
  ExperimentalWarning,
tune_num_leaves, val_score: 2.343874: 100%|##########| 20/20 [1:00:09<00:00, 180.46s/it]
tune_bagging_fraction_and_bagging_freq, val_score: 2.343874:  10%|#         | 1/10 [00:47<07:08, 47.58s/it][I 2020-05-17 08:10:12,701] Finished trial#0 with value: 2.3452018234827876 with parameters: {'bagging_fraction': 0.5576456634374, 'bagging_freq': 5}. Best is trial#0 with value: 2.3452018234827876.
tune_bagging_fraction_and_bagging_freq, val_score: 2.343874:  20%|##        | 2/10 [01:38<06:28, 48.58s/it][I 2020-05-17 08:11:03,626] Finished trial#1 with value: 2.344753657855863 with parameters: {'bagging_fraction': 0.6322524345012235, 'bagging_freq': 6}. Best is trial#1 with value: 2.344753657855863.
tune_bagging_fraction_and_bagging_freq, val_score: 2.343874:  30%|###       |

tune_bagging_fraction_and_bagging_freq, val_score: 2.343874:  40%|####      | 4/10 [03:10<04:41, 46.94s/it][I 2020-05-17 08:12:35,739] Finished trial#3 with value: 2.347535504993517 with parameters: {'bagging_fraction': 0.446479946837105, 'bagging_freq': 6}. Best is trial#2 with value: 2.3444765497369504.
tune_bagging_fraction_and_bagging_freq, val_score: 2.343463:  50%|#####     | 5/10 [04:14<04:20, 52.07s/it][I 2020-05-17 08:13:39,752] Finished trial#4 with value: 2.343463155397856 with parameters: {'bagging_fraction': 0.9018117847077181, 'bagging_freq': 5}. Best is trial#4 with value: 2.343463155397856.
tune_bagging_fraction_and_bagging_freq, val_score: 2.343426:  60%|######    | 6/10 [05:20<03:44, 56.11s/it][I 2020-05-17 08:14:45,282] Finished trial#5 with value: 2.3434260503484667 with parameters: {'bagging_fraction': 0.9117189906923387, 'bagging_freq': 4}. Best is trial#5 with value: 2.3434260503484667.
tune_bagging_fraction_and_bagging_freq, val_score: 2.343426:  70%|#######   |

tune_lambda_l1_and_lambda_l2, val_score: 2.341204:  85%|########5 | 17/20 [19:04<03:25, 68.65s/it][I 2020-05-17 08:40:42,070] Finished trial#16 with value: 2.3412213149540126 with parameters: {'lambda_l1': 7.100398044213165, 'lambda_l2': 0.010902090142405134}. Best is trial#13 with value: 2.34120379927918.
tune_lambda_l1_and_lambda_l2, val_score: 2.341204:  90%|######### | 18/20 [20:10<02:15, 67.97s/it][I 2020-05-17 08:41:48,462] Finished trial#17 with value: 2.342893434650894 with parameters: {'lambda_l1': 0.8133276357055164, 'lambda_l2': 0.010342203388564574}. Best is trial#13 with value: 2.34120379927918.
tune_lambda_l1_and_lambda_l2, val_score: 2.341204:  95%|#########5| 19/20 [21:16<01:07, 67.32s/it][I 2020-05-17 08:42:54,268] Finished trial#18 with value: 2.343226968858258 with parameters: {'lambda_l1': 0.004149434390145978, 'lambda_l2': 0.0025282656641140247}. Best is trial#13 with value: 2.34120379927918.
tune_lambda_l1_and_lambda_l2, val_score: 2.341204: 100%|##########| 20/20

Best Params: {'lambda_l1': 5.4143376259630465, 'lambda_l2': 0.006926732664484028, 'num_leaves': 4, 'feature_fraction': 0.41600000000000004, 'bagging_fraction': 0.9117189906923387, 'bagging_freq': 4, 'min_child_samples': 25}
CPU times: user 1d 6h 42min 41s, sys: 8min 7s, total: 1d 6h 50min 48s
Wall time: 1h 56min 20s


### Feature Importance

In [14]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
0,pitch_cnt_in_game,390
181,rank_x_year_dif_b-c,390
174,bmi_dif_p-c,404
113,high_str,414
122,right_ball,417
6,pitcher_cnt_in_game,417
109,total,419
115,mid_str,429
173,rank_x_year_dif_p-c,430
103,slider,438


### Course Tuning結果(23)
`
Best Params: {
    'lambda_l1': 5.4143376259630465, 
    'lambda_l2': 0.006926732664484028, 
    'num_leaves': 4, 
    'feature_fraction': 0.41600000000000004, 
    'bagging_fraction': 0.9117189906923387, 
    'bagging_freq': 4, 
    'min_child_samples': 25
}
`
Wall time: 1h 56min 20s

### Course Tuning結果(12)
`
Best Params: {
    'lambda_l1': 8.769293390201968, 
    'lambda_l2': 3.913949617576324e-05, 
    'num_leaves': 6, 
    'feature_fraction': 0.4, 
    'bagging_fraction': 0.8391111798378441, 
    'bagging_freq': 4, 
    'min_child_samples': 50
}
`
Wall time: 2h 58min 40s